# Demonstrate Transfer Learning and Tuning

Transfer Learning and Tuning demonstration on Cats and Dogs filtered dataset using <h3>MobileNetV2</h3>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Library Imports

In [1]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-07-29 19:13:14.470233: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-29 19:13:14.470253: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load the image dataset

In [ ]:
PARENT_DIR = os.environ.get("ONE_DRIVE")
DATA_DIR = 'Datasets/LargeDatasets/cats_and_dogs_filtered'
DATA_DIR = os.path.join(PARENT_DIR, DATA_DIR)
print(DATA_DIR)

In [ ]:
# Define the training and validation paths

train_dir = os.path.join(DATA_DIR, 'train')
validation_dir = os.path.join(DATA_DIR, 'validation')

# Directory for training images
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory for validation images
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Check the first five names of the cats ang dogs
cats_fnames = os.listdir(train_cats_dir)
dogs_fnames = os.listdir(validation_dogs_dir)

print(f"First 5 names of the cats training images - {cats_fnames[:5]}")
print(f"First 5 names of the dogs training images - {dogs_fnames[:5]}")

print(f"Total Training Cats Images - {len(os.listdir(train_cats_dir))}")
print(f"Total training dogs Images - {len(os.listdir(train_dogs_dir))}")
print(f"Total validation cats Images - {len(os.listdir(validation_cats_dir))}")
print(f"Total validation dogs Images - {len(os.listdir(validation_dogs_dir))}")

### Build the Image Data Generators

In [ ]:
# we will preprocess the image, before we will apply the image to model for training
# Adding data augmentation parameters

train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Note that validation data should not be augmented
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Flow training images in the batches of 20 using train datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=20, class_mode='binary', target_size=(128, 128))

# Flow validation images in the batches of 20 using test datagen generator
validation_generator = test_datagen.flow_from_directory(validation_dir, batch_size=20, class_mode='binary', target_size=(128, 128))

## Building the Model
### Loading the pre-trained model (MobileNetV2)

In [ ]:
IMAGE_SHAPE = (128, 128, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE, include_top=False, weights='imagenet')
base_model.summary()

# We would not take the top and output layer of the MobileNetV2 as it is explicitly trained for IMAGENET .. We would need to define a custom head for our specific task and hence we set it to False

### Freezing the base model

In [ ]:
# We don't want to re-train the base model again and would want to use its existing weights ,.. hence
base_model.trainable = False

# We will train only the layers which we will add for our custom task

### Defining the custom head for our network

Custom head is always designed fer the task, that we are going to solve

In [ ]:
# First thing to do is to check, what is the output size from the base model
base_model.output

In [ ]:
# This shape is not suitable for our task, we can resolve it by using Global average pooling
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
global_average_layer

Now it is converted to size (Mone, 1280) - it has kept only the last dimension

In [ ]:
# Now we will just add the output layer
prediction_layer = tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid) (global_average_layer)

### Defining the Model

In [ ]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)
model.summary()

### Compiling the Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

## Training the Model

In [ ]:
model_fit = model.fit(train_generator, epochs=5, validation_data=validation_generator)

## Transfer Learning model evaluation

In [ ]:
validation_Loss , validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss - {validation_Loss}")
print(f"Validation Accuracy = {validation_accuracy}")

In [ ]:
# We will plot the training accuracy vs validation accuracy

acc = model_fit.history['accuracy']
val_acc = model_fit.history['val_accuracy']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
# We will plot the training loss vs validation loss

loss = model_fit.history['loss']
val_loss = model_fit.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and validation Loss')
plt.legend(loc=0)
plt.figure()


plt.show()

## Fine Tuning

There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model.

The base layers are goint to learn more atomic features like shape, edges etc and the top layers get trained on more abstract and high level features like - eyes, nose etc... So fine tuning at the top layers is necessary without disturbing the bottom layers

### Unfreeze Few top layers from the Model

In [ ]:
base_model.trainable = True
print(f"Number of layers in the base model - {len(base_model.layers)}")

In [ ]:
fine_tune_at = 100 # index of the layer from which we are going to fine tune, which means we are going to fine tune all layers from 100 to 154

# remaining base layers are not to be trained
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compile the model for fine tuning

In [ ]:
# We are basically recompiling the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
#model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

### Perform the fine tuning

In [ ]:
model_fit = model.fit(train_generator, epochs=5, validation_data=validation_generator)

In [ ]:
validation_Loss , validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss - {validation_Loss}")
print(f"Validation Accuracy = {validation_accuracy}")